In [ ]:
pip install pyspark spark-nlp pandas matplotlib

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, udf, lower, trim, regexp_replace, count
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT

import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, DeBertaEmbeddings

# Khởi tạo Spark NLP
spark = sparknlp.start()

# Đọc dữ liệu từ file JSONL
input_file_path = "/opt/workspace/data.jsonl"
df = spark.read.option("multiLine", False).json(input_file_path)

# Trích xuất câu hỏi từ role là "user"
user_questions = df.select(explode("messages").alias("msg")) \
    .filter(col("msg.role") == "user") \
    .select(col("msg.content").alias("text")) \
    .filter(col("text").isNotNull())

# Tạo pipeline NLP
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = DeBertaEmbeddings.pretrained("deberta_embeddings_spm_vie", "vie") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setCaseSensitive(True)

pipeline = Pipeline(stages=[document_assembler, tokenizer, embeddings])

# Chạy pipeline
model = pipeline.fit(user_questions)
embedded_data = model.transform(user_questions)

# UDF để tính vector trung bình từ embedding
def avg_embeddings(embeddings):
    if embeddings:
        avg = np.mean([e['embeddings'] for e in embeddings], axis=0)
        return Vectors.dense(avg.tolist())
    return Vectors.dense([])

avg_embeddings_udf = udf(avg_embeddings, VectorUDT())

# Chuyển thành vector đặc trưng
vectorized_data = embedded_data.withColumn("features", avg_embeddings_udf(col("embeddings")))

# Chuẩn hóa vector
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(vectorized_data)
scaled_data = scaler_model.transform(vectorized_data)

# KMeans clustering
kmeans = KMeans(featuresCol="scaled_features", predictionCol="cluster", k=3)
kmeans_model = kmeans.fit(scaled_data)
clustered_data = kmeans_model.transform(scaled_data)

# Chuẩn hóa text: chỉ loại dấu câu, giữ nguyên tiếng Việt
normalized_data = clustered_data.withColumn(
    "normalized_text",
    trim(lower(regexp_replace(col("text"), "[\\p{Punct}]", "")))
)

# Đếm tần suất các câu hỏi sau khi normalize
grouped = normalized_data.groupBy("normalized_text", "cluster") \
    .agg(count("*").alias("frequency")) \
    .orderBy(col("frequency").desc())

# Hiển thị kết quả
grouped.show(truncate=False)

# Vẽ biểu đồ tần suất theo cụm
cluster_counts = grouped.groupBy("cluster").sum("frequency") \
    .withColumnRenamed("sum(frequency)", "count") \
    .orderBy("cluster") \
    .toPandas()

plt.figure(figsize=(8, 5))
plt.bar(cluster_counts["cluster"], cluster_counts["count"], color="teal")
plt.xlabel("Cluster ID")
plt.ylabel("Number of Questions")
plt.title("Semantic Question Clustering Frequency")
plt.xticks(cluster_counts["cluster"])
plt.tight_layout()
plt.show()
